<h1 align="center"><font color="yellow">SpanMarker for Named Entity Recognition</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [SpanMarker](https://tomaarsen.github.io/SpanMarkerNER/notebooks/spacy_integration.html#Using-SpanMarker-models-for-Named-Entity-Recognition-with-spaCy)

* [tomaarsen/SpanMarkerNER](https://github.com/tomaarsen/SpanMarkerNER/tree/main)


# Usando SpanMarker com spaCy

O [SpanMarker](https://github.com/tomaarsen/SpanMarkerNER/tree/main) é um módulo `Python` acessível, mas poderoso, para treinar modelos de Reconhecimento de Entidade Nomeada.

Neste breve caderno, veremos como usar modelos `SpanMarker` pré-treinados com [spaCy](https://spacy.io/).

# Configuração

Primeiro de tudo, ambos `spacy` e o módulo Python `span_marker` precisam ser instalados. Depois, precisamos instalar um `modelo spacy` também. Vamos escolher o mais simples por enquanto: `pt_core_news_sm` (pode usar, também, para Inglês: `en_core_web_sm`).


```
%pip install span_marker spacy

!python -m spacy download pt_core_news_sm

!spacy download en_core_web_sm
```

# Usando spaCy para Reconhecimento de Entidade Nomeada

Começaremos usando puramente `spaCy` para `NER`, para ajudar a dar uma indicação das alterações que precisam ser feitas para usar os modelos `SpanMarker` para `NER`.


In [20]:
import spacy

# Carregamos o Modelo spaCy
nlp = spacy.load("pt_core_news_sm")

# Alimente algum texto através do modelo para obter um doc spaCy
text = """Cleopatra VII, também conhecida como Cleópatra (a Grande), \
foi a última governante ativa do Reino Ptolemaico do Egito. Ela nasceu em \
69 aC e governou o Egito de 51 aC até sua morte em 30 aC"""
doc = nlp(text)

# E olhe para as Entidades:
doc.ents


(Cleopatra VII, Cleópatra, Reino Ptolemaico do Egito, Egito)

<font color="orange">O módulo `spaCy` vem com um visualizador conveniente que podemos usar para inspecionar essas entidades de uma maneira mais conveniente, vamos usá-lo.</font>

In [21]:
from spacy import displacy

displacy.render(doc, style="ent")